# Data Wrangling: Join, Combine, and Reshape

***

Author: Jiacheng

Date: 2019-05-17

***

In [1]:
import numpy as np
import pandas as pd

---

## 1 Hierarchical Indexing
## 层次化索引

* ### 1. 层次化入门

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
data

a  1    1.353601
   2   -1.321875
   3   -0.152124
b  1    0.632364
   3    1.766706
c  1    0.642660
   2   -1.518922
d  2   -0.845785
   3   -0.082289
dtype: float64

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [5]:
# 部分索引
data['b']

1    0.632364
3    1.766706
dtype: float64

In [7]:
data['b':'d']

b  1    0.632364
   3    1.766706
c  1    0.642660
   2   -1.518922
d  2   -0.845785
   3   -0.082289
dtype: float64

In [8]:
data.loc[['b', 'd']]

b  1    0.632364
   3    1.766706
d  2   -0.845785
   3   -0.082289
dtype: float64

In [9]:
# 取出各部分第二项
data.loc[:, 2]

a   -1.321875
c   -1.518922
d   -0.845785
dtype: float64

In [10]:
# 形成DataFrame
data.unstack()

1         2         3
a  1.353601 -1.321875 -0.152124
b  0.632364       NaN  1.766706
c  0.642660 -1.518922       NaN
d       NaN -0.845785 -0.082289

In [11]:
# 逆运算
data.unstack().stack()

a  1    1.353601
   2   -1.321875
   3   -0.152124
b  1    0.632364
   3    1.766706
c  1    0.642660
   2   -1.518922
d  2   -0.845785
   3   -0.082289
dtype: float64

In [12]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                             ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
# 指定index的名称
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [14]:
frame['Colorado']

color      Green
key1 key2       
a    1         2
     2         5
b    1         8
     2        11

* ### 2. 根据级别汇总统计

In [16]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [18]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

* ### 3. 使用DataFrame的列进行索引

In [5]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

frame

a  b    c  d
0  0  7  one  0
1  1  6  one  1
2  2  5  one  2
3  3  4  two  0
4  4  3  two  1
5  5  2  two  2
6  6  1  two  3

In [6]:
# 将某列转换为行索引
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [7]:
# 保留
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [8]:
frame2.reset_index()

c  d  a  b
0  one  0  0  7
1  one  1  1  6
2  one  2  2  5
3  two  0  3  4
4  two  1  4  3
5  two  2  5  2
6  two  3  6  1

---

## 2. Combining and Merging Datasets
## 合并数据集

* ### 1. 数据库风格的DataFrame合并

In [9]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [10]:
df1

key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6

In [11]:
df2

key  data2
0   a      0
1   b      1
2   d      2

In [12]:
# 求取数据集的交集
pd.merge(df1, df2)

key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0

In [13]:
pd.merge(df1, df2, on='key')

key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0

In [14]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

In [15]:
df3

lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6

In [16]:
df4

rkey  data2
0    a      0
1    b      1
2    d      2

In [17]:
# 列名不同，分别指定
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

lkey  data1 rkey  data2
0    b      0    b      1
1    b      1    b      1
2    b      6    b      1
3    a      2    a      0
4    a      4    a      0
5    a      5    a      0

In [18]:
# 求取数据集的并集
pd.merge(df1, df2, how='outer')

key  data1  data2
0   b    0.0    1.0
1   b    1.0    1.0
2   b    6.0    1.0
3   a    2.0    0.0
4   a    4.0    0.0
5   a    5.0    0.0
6   c    3.0    NaN
7   d    NaN    2.0

In [19]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

In [20]:
df1

key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5

In [21]:
df2

key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4

In [22]:
# 使用第一个表的键作为合并表的键
pd.merge(df1, df2, on='key', how='left')

key  data1  data2
0    b      0    1.0
1    b      0    3.0
2    b      1    1.0
3    b      1    3.0
4    a      2    0.0
5    a      2    2.0
6    c      3    NaN
7    a      4    0.0
8    a      4    2.0
9    b      5    1.0
10   b      5    3.0

In [23]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

key1 key2  lval  rval
0  foo  one   1.0   4.0
1  foo  one   1.0   5.0
2  foo  two   2.0   NaN
3  bar  one   3.0   6.0
4  bar  two   NaN   7.0

In [24]:
pd.merge(left, right, on='key1')

key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7

In [25]:
# 重叠列添加名字
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7

* ### 2. 索引上的合并

In [26]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [27]:
left1

key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5

In [28]:
right1

group_val
a        3.5
b        7.0

In [29]:
pd.merge(left1, right1, left_on='key', right_index=True)

key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0

In [30]:
# 求并集 -- outer
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0
5   c      5        NaN

In [31]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [32]:
lefth

key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0

In [33]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [34]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

key1  key2  data  event1  event2
0    Ohio  2000   0.0       4       5
0    Ohio  2000   0.0       6       7
1    Ohio  2001   1.0       8       9
2    Ohio  2002   2.0      10      11
3  Nevada  2001   3.0       0       1

In [35]:
pd.merge(lefth, righth, left_on=['key1'], right_index=True)
# 必须以列表的形式指明多个列

ValueError: len(left_on) must equal the number of levels in the index of "right"

In [36]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                       columns=['Missouri', 'Alabama'])

In [37]:
left2

Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0

In [38]:
right2

Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0

In [39]:
# 同时合并双方索引
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0

In [40]:
# 默认左连接
left2.join(right2, how='outer')

Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0

In [41]:
left1.join(right1, on='key')

key  value  group_val
0   a      0        3.5
1   b      1        7.0
2   a      2        3.5
3   a      3        3.5
4   b      4        7.0
5   c      5        NaN

In [42]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York','Oregon'])
another


New York  Oregon
a       7.0     8.0
c       9.0    10.0
e      11.0    12.0
f      16.0    17.0

In [43]:
left2

Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0

In [44]:
left2.join([right2, another])

Ohio  Nevada  Missouri  Alabama  New York  Oregon
a   1.0     2.0       NaN      NaN       7.0     8.0
c   3.0     4.0       9.0     10.0       9.0    10.0
e   5.0     6.0      13.0     14.0      11.0    12.0

* ### 3. 轴向连接

In [45]:
arr = np.arange(12).reshape((3, 4))

In [46]:
# 数据合并，连接（concatenation）
np.concatenate([arr, arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [48]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c','d','e'])
s3 = pd.Series([5, 6], index=['f','g'])
# 每一列为不同series
pd.concat([s1, s2, s3], axis=1)

/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0

In [49]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [50]:
pd.concat([s1, s4], axis=1)

/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


0  1
a  0.0  0
b  1.0  1
f  NaN  5
g  NaN  6

In [51]:
# 求交集
pd.concat([s1, s4], axis=1, join='inner')

0  1
a  0  0
b  1  1

In [53]:
 pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'g']])

0    1
a  0.0  0.0
c  NaN  NaN
b  1.0  1.0
g  NaN  6.0

In [54]:
result = pd.concat([s1, s1, s3], keys=['one','two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [55]:
result.unstack()

a    b    f    g
one    0.0  1.0  NaN  NaN
two    0.0  1.0  NaN  NaN
three  NaN  NaN  5.0  6.0

In [56]:
pd.concat([s1, s2, s3], axis=1, keys=['one','two', 'three'])

/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


one  two  three
a  0.0  NaN    NaN
b  1.0  NaN    NaN
c  NaN  2.0    NaN
d  NaN  3.0    NaN
e  NaN  4.0    NaN
f  NaN  NaN    5.0
g  NaN  NaN    6.0

In [57]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])


In [58]:
df1

one  two
a    0    1
b    2    3
c    4    5

In [59]:
df2

three  four
a      5     6
c      7     8

In [61]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [2]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [3]:
df1

a         b         c         d
0 -0.250186 -1.538075  0.955802  0.739287
1 -0.419705 -0.110315  0.285438  0.305727
2 -0.153436  0.333557  0.941951 -0.131349

In [4]:
df2

b         d         a
0 -0.47777 -0.091311 -0.055478
1  0.26291 -0.071524  1.232644

In [5]:
 pd.concat([df1, df2])

/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


a         b         c         d
0 -0.250186 -1.538075  0.955802  0.739287
1 -0.419705 -0.110315  0.285438  0.305727
2 -0.153436  0.333557  0.941951 -0.131349
0 -0.055478 -0.477770       NaN -0.091311
1  1.232644  0.262910       NaN -0.071524

In [6]:
 pd.concat([df1, df2], ignore_index=True)

/anaconda3/envs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


a         b         c         d
0 -0.250186 -1.538075  0.955802  0.739287
1 -0.419705 -0.110315  0.285438  0.305727
2 -0.153436  0.333557  0.941951 -0.131349
3 -0.055478 -0.477770       NaN -0.091311
4  1.232644  0.262910       NaN -0.071524

* ### 4. 合并重叠数据

In [7]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [8]:
b[-1] = np.nan

In [9]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [10]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [11]:
# np.where(condition, x, y): 满足condition,输出x, 否则输出y
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [14]:
 b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [15]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [16]:
df1

a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14

In [17]:
df2

a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0

In [18]:
df1.combine_first(df2)

a    b     c
0  1.0  NaN   2.0
1  4.0  2.0   6.0
2  5.0  4.0  10.0
3  3.0  6.0  14.0
4  7.0  8.0   NaN

---

## 3. Reshaping and Pivoting
## 重塑和轴向旋转

* ### 1. 重塑层次化索引

In [19]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                     index=pd.Index(['Ohio','Colorado'], name='state'),
                     columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5

In [21]:
# 列转为行
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [27]:
result.unstack()

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5

In [28]:
result.unstack(0)

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5

In [29]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [30]:
data2.unstack()

a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0

In [32]:
# 可逆
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [33]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [34]:
df.unstack(0)

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [38]:
# 指定轴的名字
df.unstack(0).stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

* ### 2. 将“长格式”旋转为“宽格式”

In [39]:
data = pd.read_csv('../examples/macrodata.csv')
data.head()

year  quarter   realgdp  realcons  realinv  realgovt  realdpi    cpi  \
0  1959.0      1.0  2710.349    1707.4  286.898   470.045   1886.9  28.98   
1  1959.0      2.0  2778.801    1733.7  310.859   481.301   1919.7  29.15   
2  1959.0      3.0  2775.488    1751.8  289.226   491.260   1916.4  29.35   
3  1959.0      4.0  2785.204    1753.7  299.356   484.052   1931.3  29.37   
4  1960.0      1.0  2847.699    1770.5  331.722   462.199   1955.5  29.54   

      m1  tbilrate  unemp      pop  infl  realint  
0  139.7      2.82    5.8  177.146  0.00     0.00  
1  141.7      3.08    5.1  177.830  2.34     0.74  
2  140.5      3.82    5.3  178.657  2.74     1.09  
3  140.0      4.33    5.6  179.386  0.27     4.06  
4  139.6      3.50    5.2  180.007  2.31     1.19

In [40]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item                           infl    realgdp  unemp
date                                                 
1959-03-31 23:59:59.999999999  0.00   2710.349    5.8
1959-06-30 23:59:59.999999999  2.34   2778.801    5.1
1959-09-30 23:59:59.999999999  2.74   2775.488    5.3
1959-12-31 23:59:59.999999999  0.27   2785.204    5.6
1960-03-31 23:59:59.999999999  2.31   2847.699    5.2
1960-06-30 23:59:59.999999999  0.14   2834.390    5.2
1960-09-30 23:59:59.999999999  2.70   2839.022    5.6
1960-12-31 23:59:59.999999999  1.21   2802.616    6.3
1961-03-31 23:59:59.999999999 -0.40   2819.264    6.8
1961-06-30 23:59:59.999999999  1.47   2872.005    7.0
1961-09-30 23:59:59.999999999  0.80   2918.419    6.8
1961-12-31 23:59:59.999999999  0.80   2977.830    6.2
1962-03-31 23:59:59.999999999  2.26   3031.241    5.6
1962-06-30 23:59:59.999999999  0.13   3064.709    5.5
1962-09-30 23:59:59.999999999  2.11   3093.047    5.6
1962-12-31 23:59:59.999999999  0.79   3100.563    5.5
1963-03-31 23:59:59.999999999  0.53   3141.087    5.8
1963-06-30 23:59:59.999999999  2.75   3180.447    5.7
1963-09-30 23:59:59.999999999  0.78   3240.332    5.5
1963-12-31 23:59:59.999999999  2.46   3264.967    5.6
1964-03-31 23:59:59.999999999  0.13   3338.246    5.5
1964-06-30 23:59:59.999999999  0.90   3376.587    5.2
1964-09-30 23:59:59.999999999  1.29   3422.469    5.0
1964-12-31 23:59:59.999999999  2.05   3431.957    5.0
1965-03-31 23:59:59.999999999  1.28   3516.251    4.9
1965-06-30 23:59:59.999999999  2.54   3563.960    4.7
1965-09-30 23:59:59.999999999  0.89   3636.285    4.4
1965-12-31 23:59:59.999999999  2.90   3724.014    4.1
1966-03-31 23:59:59.999999999  4.99   3815.423    3.9
1966-06-30 23:59:59.999999999  2.10   3828.124    3.8
...                             ...        ...    ...
2002-06-30 23:59:59.999999999  1.56  11538.770    5.8
2002-09-30 23:59:59.999999999  2.66  11596.430    5.7
2002-12-31 23:59:59.999999999  3.08  11598.824    5.8
2003-03-31 23:59:59.999999999  1.31  11645.819    5.9
2003-06-30 23:59:59.999999999  1.09  11738.706    6.2
2003-09-30 23:59:59.999999999  2.60  11935.461    6.1
2003-12-31 23:59:59.999999999  3.02  12042.817    5.8
2004-03-31 23:59:59.999999999  2.35  12127.623    5.7
2004-06-30 23:59:59.999999999  3.61  12213.818    5.6
2004-09-30 23:59:59.999999999  3.58  12303.533    5.4
2004-12-31 23:59:59.999999999  2.09  12410.282    5.4
2005-03-31 23:59:59.999999999  4.15  12534.113    5.3
2005-06-30 23:59:59.999999999  1.85  12587.535    5.1
2005-09-30 23:59:59.999999999  9.14  12683.153    5.0
2005-12-31 23:59:59.999999999  0.40  12748.699    4.9
2006-03-31 23:59:59.999999999  2.60  12915.938    4.7
2006-06-30 23:59:59.999999999  3.97  12962.462    4.7
2006-09-30 23:59:59.999999999 -1.58  12965.916    4.7
2006-12-31 23:59:59.999999999  3.30  13060.679    4.4
2007-03-31 23:59:59.999999999  4.58  13099.901    4.5
2007-06-30 23:59:59.999999999  2.75  13203.977    4.5
2007-09-30 23:59:59.999999999  3.45  13321.109    4.7
2007-12-31 23:59:59.999999999  6.38  13391.249    4.8
2008-03-31 23:59:59.999999999  2.82  13366.865    4.9
2008-06-30 23:59:59.999999999  8.53  13415.266    5.4
2008-09-30 23:59:59.999999999 -3.16  13324.600    6.0
2008-12-31 23:59:59.999999999 -8.79  13141.920    6.9
2009-03-31 23:59:59.999999999  0.94  12925.410    8.1
2009-06-30 23:59:59.999999999  3.37  12901.504    9.2
2009-09-30 23:59:59.999999999  3.56  12990.341    9.6

[203 rows x 3 columns]

In [42]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

date     item     value    value2
0 1959-03-31 23:59:59.999999999  realgdp  2710.349 -1.190547
1 1959-03-31 23:59:59.999999999     infl     0.000  0.105500
2 1959-03-31 23:59:59.999999999    unemp     5.800  0.595313
3 1959-06-30 23:59:59.999999999  realgdp  2778.801 -0.175658
4 1959-06-30 23:59:59.999999999     infl     2.340 -0.782500
5 1959-06-30 23:59:59.999999999    unemp     5.100  0.949151
6 1959-09-30 23:59:59.999999999  realgdp  2775.488  0.423160
7 1959-09-30 23:59:59.999999999     infl     2.740  0.773448
8 1959-09-30 23:59:59.999999999    unemp     5.300  2.059630
9 1959-12-31 23:59:59.999999999  realgdp  2785.204 -0.533702

In [43]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.105500 -1.190547   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.782500 -0.175658   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.773448  0.423160   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.009832 -0.533702   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  2.687250  0.379320   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.595313  
1959-06-30 23:59:59.999999999  0.949151  
1959-09-30 23:59:59.999999999  2.059630  
1959-12-31 23:59:59.999999999  1.466428  
1960-03-31 23:59:59.999999999 -0.173473

In [44]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [45]:
data.head()

item                            realgdp  infl  unemp
date                                                
1959-03-31 23:59:59.999999999  2710.349  0.00    5.8
1959-06-30 23:59:59.999999999  2778.801  2.34    5.1
1959-09-30 23:59:59.999999999  2775.488  2.74    5.3
1959-12-31 23:59:59.999999999  2785.204  0.27    5.6
1960-03-31 23:59:59.999999999  2847.699  2.31    5.2

In [46]:
pivoted['value'][:5]

item                           infl   realgdp  unemp
date                                                
1959-03-31 23:59:59.999999999  0.00  2710.349    5.8
1959-06-30 23:59:59.999999999  2.34  2778.801    5.1
1959-09-30 23:59:59.999999999  2.74  2775.488    5.3
1959-12-31 23:59:59.999999999  0.27  2785.204    5.6
1960-03-31 23:59:59.999999999  2.31  2847.699    5.2

In [48]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.105500 -1.190547   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.782500 -0.175658   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.773448  0.423160   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.009832 -0.533702   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  2.687250  0.379320   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -1.221144 -0.470103   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  0.332407 -0.263127   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.595313  
1959-06-30 23:59:59.999999999  0.949151  
1959-09-30 23:59:59.999999999  2.059630  
1959-12-31 23:59:59.999999999  1.466428  
1960-03-31 23:59:59.999999999 -0.173473  
1960-06-30 23:59:59.999999999 -0.904304  
1960-09-30 23:59:59.999999999  1.550825

* ### 3. 将“宽格式”旋转为“长格式”

In [49]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

key  A  B  C
0  foo  1  4  7
1  bar  2  5  8
2  baz  3  6  9

In [51]:
melted = pd.melt(df, ['key'])
melted

key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        B      4
4  bar        B      5
5  baz        B      6
6  foo        C      7
7  bar        C      8
8  baz        C      9

In [53]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable  A  B  C
key              
bar       2  5  8
baz       3  6  9
foo       1  4  7

In [54]:
reshaped.reset_index()

variable  key  A  B  C
0         bar  2  5  8
1         baz  3  6  9
2         foo  1  4  7

In [55]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'C'])

key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        C      7
4  bar        C      8
5  baz        C      9

In [56]:
pd.melt(df, value_vars=['A', 'B', 'C'])

variable  value
0        A      1
1        A      2
2        A      3
3        B      4
4        B      5
5        B      6
6        C      7
7        C      8
8        C      9

In [57]:
pd.melt(df, value_vars=['key', 'A', 'B'])

variable value
0      key   foo
1      key   bar
2      key   baz
3        A     1
4        A     2
5        A     3
6        B     4
7        B     5
8        B     6